CROWDPULSE


In [1]:
import os

# 🔐 Twilio credentials (PLACEHOLDERS ONLY)
os.environ["TWILIO_ACCOUNT_SID"] = "TWILIO_ACCOUNT_SID_HERE"
os.environ["TWILIO_AUTH_TOKEN"]  = "TWILIO_AUTH_TOKEN_HERE"
os.environ["TWILIO_NUMBER"]      = "TWILIO_PHONE_NUMBER_HERE"

# 🔐 Groq LLM
os.environ["GROQ_API_KEY"] = "GROQ_API_KEY_HERE"

# 🔐 Ngrok
os.environ["NGROK_AUTH_TOKEN"] = "NGROK_AUTH_TOKEN_HERE"

print("🔐 Placeholder keys loaded")

🔐 Placeholder keys loaded


In [5]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import cv2
import numpy as np
import json
import time
from datetime import datetime
from collections import deque
from PIL import Image
import matplotlib.pyplot as plt

# Build CSRNet
class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()
        vgg = models.vgg16(weights='DEFAULT')
        self.frontend = nn.Sequential(*list(vgg.features.children())[:23])
        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(),
            nn.Conv2d(512, 256, 3, padding=2, dilation=2), nn.ReLU(),
            nn.Conv2d(256, 128, 3, padding=2, dilation=2), nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=2, dilation=2), nn.ReLU(),
        )
        self.output_layer = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x

model = CSRNet()
print("Model built")

Model built


In [6]:
!wget https://huggingface.co/rootstrap-org/crowd-counting/resolve/main/weights.pth -O csrnet_weights.pth

checkpoint = torch.load('/content/csrnet_weights.pth', map_location='cpu')
model.load_state_dict(checkpoint)
model.eval()
print("Weights loaded")

--2026-02-28 11:12:49--  https://huggingface.co/rootstrap-org/crowd-counting/resolve/main/weights.pth
Resolving huggingface.co (huggingface.co)... 3.165.160.59, 3.165.160.11, 3.165.160.12, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/65a7044c293f73160dd702a4/950644f6b69af20d5cea617c94a6c18734cebcba84dac27bc73e9956c4151137?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27weights.pth%3B+filename%3D%22weights.pth%22%3B&Expires=1772280769&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzcyMjgwNzY5fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjVhNzA0NGMyOTNmNzMxNjBkZDcwMmE0Lzk1MDY0NGY2YjY5YWYyMGQ1Y2VhNjE3Yzk0YTZjMTg3MzRjZWJjYmE4NGRhYzI3YmM3M2U5OTU2YzQxNTExMzdcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=gpZZ5v9RAA4sZcyE8II9uQVMIdqhdk15CZxum67cFD2fXfOHrlHIuHuiqym-g9tyL4

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Zone history
zone_history = {
    "Zone_A": deque(maxlen=5),
    "Zone_B": deque(maxlen=5),
    "Zone_C": deque(maxlen=5),
    "Zone_D": deque(maxlen=5)
}

def get_status(score):
    if score > 200:
        return "CRITICAL", (0, 0, 255)
    elif score > 150:
        return "DANGER",   (0, 0, 200)
    elif score > 100:
        return "WARNING",  (0, 165, 255)
    elif score > 50:
        return "MONITOR",  (0, 255, 255)
    else:
        return "SAFE",     (0, 255, 0)

def predict_time_to_danger(history, threshold=200):
    values = list(history)
    if len(values) < 2:
        return None
    growth_rates = [values[i+1] - values[i] for i in range(len(values)-1)]
    avg_growth = sum(growth_rates) / len(growth_rates)
    if avg_growth <= 0:
        return None
    current = values[-1]
    if current >= threshold:
        return 0
    return round((threshold - current) / avg_growth, 1)

def process_frame(frame):
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        output = model(img_tensor)
    density_map = output.squeeze().detach().numpy()
    h, w = density_map.shape
    zones_data = {
        "Zone_A": float(density_map[0:h//2, 0:w//2].sum()),
        "Zone_B": float(density_map[0:h//2, w//2:w].sum()),
        "Zone_C": float(density_map[h//2:h, 0:w//2].sum()),
        "Zone_D": float(density_map[h//2:h, w//2:w].sum())
    }
    for zone, score in zones_data.items():
        zone_history[zone].append(score)
    return zones_data

def draw_zones_on_frame(frame, zones_data):
    h, w = frame.shape[:2]
    zone_coords = {
        "Zone_A": (0, 0, w//2, h//2),
        "Zone_B": (w//2, 0, w, h//2),
        "Zone_C": (0, h//2, w//2, h),
        "Zone_D": (w//2, h//2, w, h)
    }
    for zone, (x1, y1, x2, y2) in zone_coords.items():
        score = zones_data[zone]
        status, color = get_status(score)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        cv2.putText(frame, f"{zone}: {status} ({score:.0f})",
                    (x1+10, y1+35), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, color, 2)
    return frame

def generate_alert_json(zone_history, zones_data):
    output = {
        "timestamp": datetime.now().isoformat(),
        "total_count": int(sum(zones_data.values())),
        "zones": {},
        "active_alerts": []
    }
    for zone, history in zone_history.items():
        current = list(history)[-1] if history else 0
        mins = predict_time_to_danger(history)
        status, _ = get_status(current)
        alert = status in ["CRITICAL", "DANGER", "WARNING"]
        output["zones"][zone] = {
            "current_count": round(current, 1),
            "status": status,
            "predicted_danger_in_mins": mins,
            "alert": alert
        }
        if alert:
            output["active_alerts"].append({
                "zone": zone,
                "message": f"{zone} crowd at {status} level — {f'danger in {mins} mins' if mins and mins > 0 else 'IMMEDIATE ACTION NEEDED'}",
                "urgency": status
            })
    return output

print("All functions ready")

All functions ready


In [8]:
from google.colab import files
print("Upload your crowd video")
uploaded_video = files.upload()
video_path = list(uploaded_video.keys())[0]
print(f"Uploaded: {video_path}")

Upload your crowd video


Saving Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube_720p.mp4 to Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube_720p (1).mp4
Uploaded: Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube_720p (1).mp4


Making phone call alerts

In [9]:
!pip install twilio
print("Twilio installed")

Twilio installed


In [10]:
from twilio.rest import Client
import os
from datetime import datetime

# Read Twilio credentials from environment variables
ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")
AUTH_TOKEN  = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_NUMBER = os.getenv("TWILIO_NUMBER")

if not all([ACCOUNT_SID, AUTH_TOKEN, TWILIO_NUMBER]):
    raise ValueError("❌ Twilio environment variables not set")

# Guard registry — which guard covers which zone
guards = {
    "Zone_A": {"name": "Bene",  "phone": "+919148910903"},
    "Zone_B": {"name": "Rohi",  "phone": "+918754787789"},
    "Zone_C": {"name": "Adi",   "phone": "+919148910903"},
    "Zone_D": {"name": "Ezhil", "phone": "+919148910903"}
}

# Initialize Twilio client
client = Client(ACCOUNT_SID, AUTH_TOKEN)
print("✅ Twilio client ready")
print(f"Guards registered: {len(guards)}")

✅ Twilio client ready
Guards registered: 4


Groq AI

In [11]:
!pip install -q groq

In [12]:
!pip install groq

from groq import Groq
import os

# Read Groq API key from environment
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY:
    raise ValueError("❌ GROQ_API_KEY not set")

groq_client = Groq(api_key=GROQ_API_KEY)
print("✅ Groq client ready")

✅ Groq client ready


In [13]:
# Just run this to update model name
def generate_alert_message(zone, guard_name, current_count, mins, safe_zone):
    prompt = f"""
    You are CrowdPulse, an AI safety system at a public event.
    Generate a short, urgent voice alert message for a security guard.

    Situation:
    - Zone: {zone}
    - Guard name: {guard_name}
    - Current crowd count: {current_count}
    - Minutes until critical: {mins}
    - Nearest safe zone: {safe_zone}

    Rules:
    - Max 3 sentences
    - Mention guard name, zone, time, and safe zone
    - Urgent but clear tone
    - No special characters
    """
    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

def generate_incident_report(json_data):
    prompt = f"""
    You are CrowdPulse AI. Generate a professional incident report from this crowd data:
    {json.dumps(json_data, indent=2)}

    Include:
    - Summary with exact crowd counts per zone
    - Which zones were dangerous and their predicted_danger_in_mins value
    - Example: "Zone C had 133 people — critical predicted in 10.6 minutes"
    - Actions taken
    - Recommendations specific to these zones

    Keep under 150 words. Use the exact numbers from the data.
    """
    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

def generate_crowd_analysis(all_json_outputs):
    prompt = f"""
    You are CrowdPulse AI, an expert crowd safety analyst for Indian public events.
    Analyze this real sensor data from a live event:
    {json.dumps(all_json_outputs, indent=2)}

    Give SPECIFIC, DATA-DRIVEN insights only. No generic advice.
    Use the actual zone names, timestamps, and crowd counts from the data.

    Format your response exactly like this:

    PEAK CONGESTION:
    [exact zone, exact time, exact count from data]

    DANGER TIMELINE:
    [list each zone alert with exact time it triggered and how long it lasted]

    CROWD FLOW PATTERN:
    [what actually happened zone by zone based on the numbers]

    TOP 3 RECOMMENDATIONS FOR NEXT EVENT:
    1. [specific to this venue's zones and what actually went wrong]
    2. [specific gate/timing recommendation based on peak time observed]
    3. [specific staffing recommendation based on which zones were problematic]

    Use only data from the JSON. No assumptions. No generic statements.
    Keep under 200 words.
    """
    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [14]:
def make_genai_alert_call(zone, guard_name, guard_phone, current_count, mins):

    # Find nearest safe zone
    safe_zones = {"Zone_A": "Zone_B", "Zone_B": "Zone_A",
                  "Zone_C": "Zone_D", "Zone_D": "Zone_C"}
    safe_zone = safe_zones.get(zone, "nearest exit")

    # Generate AI message
    ai_message = generate_alert_message(
        zone=zone,
        guard_name=guard_name,
        current_count=current_count,
        mins=mins,
        safe_zone=safe_zone
    )

    print(f"AI Generated Message: {ai_message}")

    # Fire voice call with AI message
    twiml_message = f"""
    <Response>
        <Say voice="alice" language="en-IN">
            {ai_message}
        </Say>
        <Pause length="1"/>
        <Say voice="alice" language="en-IN">
            Repeating. {ai_message}
        </Say>
    </Response>
    """

    try:
        call = client.calls.create(
            twiml=twiml_message,
            to=guard_phone,
            from_=TWILIO_NUMBER
        )
        print(f"AI Alert call fired to {guard_name}")
        print(f"Call SID: {call.sid}")
        return True
    except Exception as e:
        print(f"Call failed: {e}")
        return False

print("GenAI + Voice call connected")

GenAI + Voice call connected


Backend

In [15]:
!pip install flask flask-cors pyngrok

In [16]:
# Initially run this once. And later if u wanted to try with new video run this and it clears the old paths, so from top run all cells again.
# This is just to connect my local VsCode (frontend) with Colab (backend)
# This cell prepares Colab backend for frontend connection
import os
import subprocess
from pyngrok import ngrok

# Safely free port 5050 (Colab/Linux)
subprocess.run("fuser -k 5050/tcp", shell=True, capture_output=True)

# Read ngrok auth token from environment
NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")

if not NGROK_AUTH_TOKEN:
    raise ValueError("❌ NGROK_AUTH_TOKEN not set")

# Reset ngrok
ngrok.kill()
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("✅ Ngrok ready")

✅ Ngrok ready


In [17]:
# ============================================================
# CrowdPulse — FINAL LIVE LOOP
# Groq (Ideas 1, 2, 3) + Twilio + Flask API all in one cell
# ============================================================

from IPython.display import clear_output
import time
from datetime import datetime

# ── GLOBAL STATE (must be before Flask) ─────────────────────
crowd_analysis = {}
alerted_zones = {}
incident_reports = []
COOLDOWN_MINS = 5

# ── Flask API (starts in background) ────────────────────────
from flask import Flask, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading

app = Flask(__name__)
CORS(app)

@app.route("/analysis", methods=["GET"])
def get_analysis():
    if crowd_analysis:
        return jsonify(crowd_analysis)
    return jsonify({"message": "No analysis yet"}), 204

@app.route("/status", methods=["GET"])
def get_status_api():
    if json_outputs:
        return jsonify(json_outputs[-1])
    return jsonify({"message": "No data yet"}), 204

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "CrowdPulse running"})

@app.route("/reports", methods=["GET"])
def get_reports():
    return jsonify(incident_reports[-5:])

def run_flask():
    app.run(port=5050, use_reloader=False)

threading.Thread(target=run_flask, daemon=True).start()
public_url = ngrok.connect(5050)

print(f"✅ API LIVE: {public_url}/status")
print(f"📋 Reports: {public_url}/reports")

# ── Helper: check cooldown ──────────────────────────────────
def is_on_cooldown(zone):
    last = alerted_zones.get(zone)
    if not last:
        return False
    mins_since = (datetime.now() - last).seconds / 60
    return mins_since < COOLDOWN_MINS

# ── Main alert handler ──────────────────────────────────────
def handle_alert(zone, zone_data, full_json):

    if is_on_cooldown(zone):
        print(f"  ⏳ {zone} on cooldown, skipping")
        return

    guard = guards.get(zone)
    if not guard:
        print(f"  ⚠️ No guard registered for {zone}")
        return

    urgency = zone_data["status"]
    count   = zone_data["current_count"]
    mins    = zone_data["predicted_danger_in_mins"]

    safe_zones = {
        "Zone_A": "Zone_B",
        "Zone_B": "Zone_A",
        "Zone_C": "Zone_D",
        "Zone_D": "Zone_C"
    }
    safe_zone = safe_zones.get(zone, "nearest exit")

    print(f"\n  🚨 [{urgency}] {zone} — calling {guard['name']}...")

    try:
        ai_message = generate_alert_message(
            zone=zone,
            guard_name=guard["name"],
            current_count=int(count),
            mins=mins,
            safe_zone=safe_zone
        )
        print(f"  🤖 Groq message: {ai_message[:80]}...")
    except Exception as e:
        ai_message = (
            f"Alert for {guard['name']}. {urgency} in {zone}. "
            f"{int(count)} people detected. Please respond immediately."
        )
        print(f"  ⚠️ Groq failed, using fallback: {e}")

    try:
        success = make_genai_alert_call(
            zone=zone,
            guard_name=guard["name"],
            guard_phone=guard["phone"],
            current_count=int(count),
            mins=mins
        )
        if success:
            alerted_zones[zone] = datetime.now()
    except Exception as e:
        print(f"  ❌ Alert call failed: {e}")

    try:
        report = generate_incident_report(full_json)
        incident_reports.append({
            "timestamp": datetime.now().isoformat(),
            "zone": zone,
            "urgency": urgency,
            "report": report
        })
        print("  📋 Incident report saved")
    except Exception as e:
        print(f"  ⚠️ Report generation failed: {e}")

# ── MAIN VIDEO PROCESSING (RUN ONCE) ────────────────────────
for key in zone_history:
    zone_history[key].clear()

json_outputs = []
frame_count = 0

print("\n" + "=" * 50)
print("CrowdPulse LIVE — watching for danger...")
print("=" * 50)

cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    if frame_count % 30 == 0:
        zones_data = process_frame(frame)
        frame = draw_zones_on_frame(frame, zones_data)

        if all(len(h) >= 2 for h in zone_history.values()):
            result = generate_alert_json(zone_history, zones_data)
            json_outputs.append(result)

            for alert in result["active_alerts"]:
                zone = alert["zone"]
                zone_data = result["zones"][zone]
                handle_alert(zone, zone_data, result)

            clear_output(wait=True)
            print(f"✅ API: {public_url}/status")
            print(
                f"Frame {frame_count} | "
                f"Total: {result['total_count']} | "
                f"Alerts: {len(result['active_alerts'])}"
            )

            for z, d in result["zones"].items():
                status = d["status"]
                count  = int(d["current_count"])
                mins   = d["predicted_danger_in_mins"]
                icon = (
                    "🔴" if status in ["DANGER", "CRITICAL"]
                    else "🟡" if status == "WARNING"
                    else "🟢"
                )
                print(
                    f"  {icon} {z}: {status} ({count} people)"
                    f"{f' — critical in {mins} min' if mins else ''}"
                )

cap.release()

# ── GROQ IDEA 2: END-OF-VIDEO ANALYSIS (ONCE) ───────────────
if json_outputs:
    try:
        analysis_text = generate_crowd_analysis(json_outputs[-20:])
        crowd_analysis.update({
            "text": analysis_text,
            "timestamp": datetime.now().isoformat(),
            "snapshot_count": len(json_outputs)
        })
        print("📊 Analysis saved!")
    except Exception as e:
        print(f"Analysis failed: {e}")

print("✅ Video processing complete.")

✅ API: NgrokTunnel: "https://sigrid-sirenic-uniridescently.ngrok-free.dev" -> "http://localhost:5050"/status
Frame 90 | Total: 22 | Alerts: 0
  🟢 Zone_A: SAFE (8 people) — critical in 667.9 min
  🟢 Zone_B: SAFE (8 people)
  🟢 Zone_C: SAFE (1 people) — critical in 426.5 min
  🟢 Zone_D: SAFE (3 people) — critical in 199.0 min


KeyboardInterrupt: 